# Google Drive Data Reporting and Visualization Notebook

This notebook connects to Google Drive, reads data from a Google Sheet, and generates a sample report and visualization. Follow the steps below to get started.

## Step 1: Install Necessary Libraries

In [ ]:
!pip install gspread google-auth-oauthlib pandas openpyxl

## Step 2: Authenticate and Mount Google Drive

Run the following cell and follow the link to authorize access to your Google Drive. You'll need to copy the authorization code and paste it into the input box.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Step 3: Authenticate with Google Sheets API and Load Data

This section will authenticate with the Google Sheets API and attempt to load your specified Excel file from Google Drive into a pandas DataFrame. **Remember to replace `REPORT_DATE` with your desired date.**

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
import os

creds, _ = default()
gc = gspread.authorize(creds)

# --- Configuration ---
REPORT_DATE = "2025-08-07"  # <--- SET YOUR DESIRED REPORT DATE HERE (YYYY-MM-DD)

# Construct the full path to the file in Google Drive
# Adjust this path if your folder structure is different
folder_path = '/content/drive/My Drive/PTA 2025-2026 SHARED FOLDER/Subcommittees/OnVolunteers/reports/parking-hours'
file_name = f'parking-hours-{REPORT_DATE}.xlsx'
full_file_path = os.path.join(folder_path, file_name)

print(f"Attempting to load file: {full_file_path}")

try:
    # Check if the file exists in the mounted Drive
    if not os.path.exists(full_file_path):
        raise FileNotFoundError(f"File not found at {full_file_path}")

    # Read the Excel file into a pandas DataFrame
    # openpyxl is required for .xlsx files
    df = pd.read_excel(full_file_path)
    print(f"Successfully loaded {len(df)} rows from '{file_name}'.")
    print("Data Head:")
    print(df.head())

except FileNotFoundError as e:
    print(f"Error: {e}. Please ensure the file exists and Google Drive is mounted correctly.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")